# Graphes - BFS et Dijkstra

Comment trouver le chemin le plus court entre deux points? C'est la question de base en algo de graphes.

BFS (Breadth-First Search) c'est l'algo de base. Dijkstra c'est quand les chemins ont des poids differents.

---

## C'est quoi un graphe?

Des noeuds relies par des aretes. C'est tout.

Exemples:
- Reseau social: personnes = noeuds, amities = aretes
- Carte routiere: villes = noeuds, routes = aretes
- Labyrinthe: cases = noeuds, passages = aretes

In [ ]:
from collections import defaultdict

# On represente un graphe par liste d'adjacence
# Chaque noeud pointe vers ses voisins

graphe = defaultdict(list)
graphe["A"].append("B")
graphe["A"].append("C")
graphe["B"].append("D")
graphe["C"].append("D")
graphe["D"].append("E")

# Afficher
for noeud, voisins in graphe.items():
    print(f"{noeud} -> {voisins}")

Ca ressemble a ca:
```
    A
   / \
  B   C
   \ /
    D
    |
    E
```

---

## BFS - Breadth First Search

L'idee: on explore d'abord tous les voisins directs, puis les voisins des voisins, etc.

Comme une vague qui se propage.

Ca garantit de trouver le chemin le plus court (en nombre d'aretes).

In [ ]:
from collections import deque

def bfs(start, goal, graphe):
    """Trouve le plus court chemin de start a goal."""
    
    # File d'attente: (noeud, distance depuis le depart)
    queue = deque([(start, 0)])
    
    # Noeuds deja visites (pour pas tourner en rond)
    visited = {start}
    
    while queue:
        noeud, dist = queue.popleft()  # On prend le premier de la file
        
        print(f"  Visite {noeud} (distance {dist})")
        
        # Arrive!
        if noeud == goal:
            return dist
        
        # Ajouter les voisins non visites
        for voisin in graphe[noeud]:
            if voisin not in visited:
                visited.add(voisin)
                queue.append((voisin, dist + 1))
    
    return -1  # Pas de chemin

print("Recherche A -> E:")
d = bfs("A", "E", graphe)
print(f"Distance: {d}")

---

## Exercice: Labyrinthe

Trouve le chemin le plus court de S a E.

In [ ]:
laby = """
S...#....
###.#.###
....#....
.###.###.
.......E.
""".strip()

grille = [list(row) for row in laby.split("\n")]
for row in grille:
    print(''.join(row))

In [ ]:
from collections import deque

def solve_maze(grille):
    rows, cols = len(grille), len(grille[0])
    
    # Trouver S et E
    for r in range(rows):
        for c in range(cols):
            if grille[r][c] == 'S': start = (r, c)
            if grille[r][c] == 'E': end = (r, c)
    
    # BFS
    directions = [(-1,0), (1,0), (0,-1), (0,1)]  # haut, bas, gauche, droite
    queue = deque([(start, 0)])
    visited = {start}
    
    while queue:
        (r, c), dist = queue.popleft()
        
        if (r, c) == end:
            return dist
        
        for dr, dc in directions:
            nr, nc = r + dr, c + dc
            
            # Check bornes + pas un mur + pas deja visite
            if 0 <= nr < rows and 0 <= nc < cols:
                if grille[nr][nc] != '#' and (nr, nc) not in visited:
                    visited.add((nr, nc))
                    queue.append(((nr, nc), dist + 1))
    
    return -1

print(f"Plus court chemin: {solve_maze(grille)} pas")

---

## Dijkstra - Quand les chemins ont des poids

BFS marche quand toutes les aretes ont le meme "cout". 

Mais si certains chemins coutent plus cher que d'autres (ex: autoroute vs chemin de terre), faut utiliser Dijkstra.

In [ ]:
import heapq

def dijkstra(start, goal, graphe):
    """graphe[noeud] = [(voisin, poids), ...]"""
    
    # File de priorite: (distance, noeud)
    # heapq garde toujours le plus petit en premier
    pq = [(0, start)]
    
    # Meilleure distance connue vers chaque noeud
    distances = {start: 0}
    
    while pq:
        dist, noeud = heapq.heappop(pq)
        
        if noeud == goal:
            return dist
        
        # Si on a deja trouve un meilleur chemin, skip
        if dist > distances.get(noeud, float('inf')):
            continue
        
        for voisin, poids in graphe[noeud]:
            new_dist = dist + poids
            
            # Si c'est mieux que ce qu'on connaissait
            if new_dist < distances.get(voisin, float('inf')):
                distances[voisin] = new_dist
                heapq.heappush(pq, (new_dist, voisin))
    
    return -1

In [ ]:
# Graphe avec poids
g = defaultdict(list)
g["A"].append(("B", 1))   # A -> B coute 1
g["A"].append(("C", 4))   # A -> C coute 4
g["B"].append(("C", 2))   # B -> C coute 2
g["B"].append(("D", 5))   # B -> D coute 5
g["C"].append(("D", 1))   # C -> D coute 1

# Le chemin direct A->B->D coute 1+5=6
# Mais A->B->C->D coute 1+2+1=4

print(f"Distance A->D: {dijkstra('A', 'D', g)}")

---

## En cyber

Les graphes c'est partout:
- Cartographie reseau (nmap, traceroute)
- Analyse de malware (graphe d'appels)
- OSINT (relations entre entites)
- Routage (BGP, OSPF)

---

## Resume

| Algo | Quand | Complexite |
|------|-------|------------|
| BFS | Tous les chemins ont le meme cout | O(V + E) |
| Dijkstra | Chemins avec poids differents | O((V+E) log V) |

V = nombre de noeuds, E = nombre d'aretes